In [ ]:
# only run cell if on google colab
!pip install pyspark
!wget https://jdbc.postgresql.org/download/postgresql-42.6.0.jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=77293ff7c09bd1a8c2f9d5aaf4799ef618560e556aa4e9694821d85dd0f495fa
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
--2023-10-11 18:34:17--  https://jdbc.postgresql.org/download/postgresql-42.6.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1.0M) [application/java-archive]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1.03M  4.96MB/s    in 0.2s    

2023-10-11 18:34:18 (4.96 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, lit
from pyspark import SparkContext, SQLContext, SparkConf

ENV = None
def is_running_in_colab():
    try:
      from google.colab import drive
      return True
    except ImportError:
      return False

# for Colab, put unzipped FIFA dataset at the root of your Google Drive
if is_running_in_colab():
  ENV = "cloud"
  print("Env: Cloud")
  data_path = "/content/drive/MyDrive/fifa_dataset"

# for local env, put unzipped FIFA dataset at same level as notebook
else:
  ENV = "local"
  print("Env: local")
  data_path = "fifa_dataset"

files_in_folder = os.listdir(data_path)

male_player_files = []
for file in files_in_folder:
  if file[0] == "p":
    male_player_files.append(file)

Env: Cloud


In [ ]:
appName = "FIFA Dataset Ingestion"
master = "local"

# Create Configuration object for Spark.
conf = SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession.builder.getOrCreate()



/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
master_df = None
for file in male_player_files:
  filepath = f"{data_path}/{file}"
  current_df = spark.read.csv(filepath, header=True, inferSchema=True)
  year = f"20{file.split('_')[-1].split('.')[0]}"
  current_df = current_df.withColumn("year", lit(year).cast("int"))
  if not master_df:
    master_df = current_df
  else:
    master_df = master_df.union(current_df)

master_df = master_df.withColumn("unique_id", monotonically_increasing_id())

master_df.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: date (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)
 

In [ ]:
if ENV == "local":
#adding combined player data to Postgres DB
  db_properties={}
  db_properties['username']="postgres"
  db_properties['password']="bigdata"
  db_properties['url']= "jdbc:postgresql://localhost:5432/FIFA_project"
  db_properties['table']="fifa"
  db_properties['driver']="org.postgresql.Driver"

  master_df.write.format("jdbc")\
  .mode("overwrite")\
  .option("url", db_properties['url'])\
  .option("dbtable", db_properties['table'])\
  .option("user", db_properties['username'])\
  .option("password", db_properties['password'])\
  .option("driver", db_properties['driver'])\
  .option("isolationLevel", "NONE")\
  .save()

else:
  # write GCP PostgreSQL connection code here
  pass

In [ ]:
from pyspark.sql.functions import count, desc

def two_one(X):
  # only using players listed in the 2022 dataset
  df_2022 = master_df.filter(master_df['year'] == 2022)

  # filter for players with contract valid until 2023
  df_23 = df_2022.filter(df_2022['club_contract_valid_until'] == 2023)

  # find X clubs with highest number of players that fulfill conditions
  x_res = df_23.groupBy("club_name").agg(count("*").alias("contract_count"))
  x_res = x_res.orderBy(desc("contract_count"))
  x_res.show(X)

two_one(5)

+--------------------+--------------+
|           club_name|contract_count|
+--------------------+--------------+
|En Avant de Guingamp|            19|
| Club Atlético Lanús|            17|
|       Lechia Gdańsk|            17|
|            Barnsley|            16|
|        Kasimpaşa SK|            16|
|        Bengaluru FC|            16|
|              Al Tai|            15|
|            KAA Gent|            15|
|      Zagłębie Lubin|            15|
|        FC Barcelona|            15|
|          CA Osasuna|            15|
|  SV Wehen Wiesbaden|            15|
|       Coventry City|            15|
|San Lorenzo de Al...|            14|
|          1. FC Köln|            14|
|           SV Meppen|            14|
|      Crystal Palace|            14|
|         Perth Glory|            14|
|  Real Valladolid CF|            14|
|Brighton & Hove A...|            14|
+--------------------+--------------+
only showing top 20 rows



In [ ]:
# y clubs w highest avg players 27+
from pyspark.sql.functions import col, avg, when,sum, countDistinct, desc, dense_rank
from pyspark.sql.window import Window

def two_two(Y):

  # create column denoting whether a players age is >= 27
  df_27 = master_df.withColumn("27+",when(col("age")>27,1).otherwise(0))

  # sum up
  y_res = df_27.groupBy("year","club_name").agg(sum("27+").alias("27+"))
  y_res = y_res.groupBy('club_name').agg(sum("27+") / countDistinct("year"))
  y_res = y_res.withColumnRenamed('(sum(27+) / count(year))', "avg_27+")
  y_res = y_res.orderBy(desc("avg_27+"))
  y_res = y_res.filter(col("club_name").isNotNull())

  # grab top Y clubs with the highest average age over 27
  window_spec = Window.partitionBy().orderBy(desc("avg_27+"))
  y_res = y_res.withColumn("rank",dense_rank().over(window_spec))
  top_y = y_res.filter(col("rank") <= Y)

  top_y.show()

two_two(5)

+--------------------+-------+----+
|           club_name|avg_27+|rank|
+--------------------+-------+----+
|  Dorados de Sinaloa|   19.0|   1|
| Matsumoto Yamaga FC|   19.0|   1|
| Shanghai Shenhua FC|   18.5|   2|
|          Qingdao FC|   18.0|   3|
|Club Deportivo Jo...|   17.5|   4|
|            Altay SK|   17.0|   5|
|         Guaireña FC|   17.0|   5|
+--------------------+-------+----+



In [ ]:
# most frequent nation position by year
from pyspark.sql.functions import max, count


freq_np = master_df.groupBy("year", "nation_position").agg(count("*"))
freq_np = freq_np.orderBy("year",col("count(1)").desc())
freq_np = freq_np.filter(col("nation_position").isNotNull())
freq_np_res = freq_np.groupBy('year').agg(max("nation_position").alias("freq_pos"))
freq_np_res.show()



+----+--------+
|year|freq_pos|
+----+--------+
|2015|     SUB|
|2016|     SUB|
|2017|     SUB|
|2018|     SUB|
|2019|     SUB|
|2020|     SUB|
|2021|     SUB|
|2022|     SUB|
+----+--------+

